In [1]:
!pip install --quiet --upgrade openai tenacity pandas ipywidgets


In [2]:
import os, time, json, math
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown

from llm_utils import chat_completion, TOKENS_USED,client
from typing import List, Dict



In [3]:
# === 2. 定义 UI 元素 ===
desc_input = widgets.Textarea(
    value="An aluminum gear with 20 teeth and a central bore.",
    placeholder="Describe the part here...",
    description="Part:",
    layout=widgets.Layout(width="100%", height="100px")
)

material_selector = widgets.Dropdown(
    options=['aluminum', 'steel'],
    value='aluminum',
    description='Material:',
    layout=widgets.Layout(width='30%')
)


generate_button = widgets.Button(
    description=" Generate CNC Plan",
    button_style="success",
    layout=widgets.Layout(width="30%", margin="10px 0")
)

output_area = widgets.Output()


In [4]:
def get_outline(part: str, material: str) -> List[Dict[str, str]]:
    """
    调用 LLM 生成加工步骤大纲，返回格式为 List[Dict]，每个元素格式为 {"step": "xxx"}。
    """
    prompt = (
        f"You are a manufacturing assistant. Given the part and material, list the high-level process steps.\n"
        f"The part is: {part}\n"
        f"The material is: {material}\n"
        "Feed must be between 100 and 1500 mm/min. "
        "Return only a JSON array of steps, where each step is an object with a 'step' field.\n"
        "Example: [{\"step\": \"Material Selection\"}, {\"step\": \"Rough Machining\"}]\n"
        "Do not explain. Do not include markdown. No commentary."
    )

    try:
        response = chat_completion(prompt,verbose=False)
        #print(" get_outline 原始返回:\n", response)  # 打印原始输出用于调试
        return json.loads(response)
    except Exception as e:
        #print(" get_outline 调用失败：", str(e))
        return []



def get_detail(outline: str | list, part: str, material: str) -> str:
    """
    构造详细 prompt，调用 LLM 生成 JSON 格式的加工步骤。
    """
    # 如果 outline 是 list[dict]，转成清晰的字符串表示
    if isinstance(outline, list):
        outline_text = "\n".join(f"- {step['step']}" for step in outline if 'step' in step)
    else:
        outline_text = outline  # 如果已经是字符串就直接使用

    prompt = (
    f"The part is: {part}\n"
    f"Material: {material}\n"
    f"Here is the outline of steps:\n"
    f"{outline_text}\n"
    "For non-machining steps (e.g. Material Selection, Quality Inspection), "
    "set rpm and feed to 0.\n"
    "Only return a valid JSON array. No explanations. No markdown."
    )

    return chat_completion(prompt)

In [5]:
def parse_llm_output(raw_json):
    data = json.loads(raw_json)
    df = pd.DataFrame(data)
    df.rename(columns={
        "rpm": "Spindle Speed (RPM)",
        "feed": "Feed Rate (mm/min)"
    }, inplace=True)
    return df


In [6]:
MATERIAL_LIMITS = {
    "aluminum": {"rpm": (3000, 12000), "feed": (800, 1500)},
    "steel":    {"rpm": (500, 1500),   "feed": (100, 300)},
    "brass":    {"rpm": (1500, 6000),  "feed": (400, 800)}
}

def validate_plan(df, material):
    rpm_min, rpm_max = MATERIAL_LIMITS[material]["rpm"]
    feed_min, feed_max = MATERIAL_LIMITS[material]["feed"]
    df["RPM Valid"]  = df["Spindle Speed (RPM)"].between(rpm_min*0.9, rpm_max*1.1)
    df["Feed Valid"] = df["Feed Rate (mm/min)"].between(feed_min*0.9, feed_max*1.1)
    return df


In [7]:
def display_plan_table(df):
    def highlight_invalid(v):
        return "background-color:#FFD2D2" if v is False else ""
    styled = df.style.map(highlight_invalid, subset=["RPM Valid","Feed Valid"])
    display(Markdown("### CNC Process Plan"))
    display(styled)


In [8]:
# === 工艺总结 ===
def reflect_summary(raw_json: str, validated_df: pd.DataFrame):
    num_steps = len(validated_df)
    num_invalid_rpm   = (~validated_df["RPM Valid"]).sum()
    num_invalid_feed  = (~validated_df["Feed Valid"]).sum()

    comment = (
        "### Reflection Summary\n"
        f"- **Total Steps Generated**: {num_steps}\n"
        f"- **Invalid Spindle Speeds**: {num_invalid_rpm} step(s)\n"
        f"- **Invalid Feed Rates**: {num_invalid_feed} step(s)\n"
    )

    if num_invalid_rpm or num_invalid_feed:
        comment += "- **🔧 Human Oversight Needed**:\n"
        if num_invalid_rpm:
            comment += "  - Some spindle speeds out of range.\n"
        if num_invalid_feed:
            comment += "  - Some feed rates out of range.\n"
    else:
        comment += "- All parameters are within expected machining constraints.\n"

    # 统计 token
    from llm_utils import TOKENS_USED
    comment += f"- **Tokens used so far**: {TOKENS_USED}\n"

    display(Markdown(comment))


In [9]:
# === 4. 按钮点击逻辑 ===

def on_generate_clicked(b):
    with output_area:
        output_area.clear_output()

        part = desc_input.value.strip()
        material = material_selector.value

        if not part:
            print("请描述你的零件。")
            return

        # 第一步：生成 outline 列表 [{"step": "..."}]
        outline = get_outline(part, material)
        if not outline:
            print("获取大纲失败，请检查网络或 API Key。")
            return

        # 第二步：使用 outline 获取详细工艺计划（JSON 字符串）
        raw_json = get_detail(outline, part, material)

        # 第三步：解析 JSON 输出为 DataFrame
        
       # ① 解析 → 得到完整工艺
        df_full = parse_llm_output(raw_json).reset_index(drop=True)
        df_full_valid = validate_plan(df_full, material=material)

        # ② 先展示【完整流程】
        display(Markdown("### Full Process Plan (incl. non-machining)"))
        display_plan_table(df_full_valid)

        # ③ 从完整表中过滤出“真正切削步骤”
        df_cut = df_full[
            df_full["Spindle Speed (RPM)"].gt(0) &
            df_full["Feed Rate (mm/min)"].gt(0)
        ].reset_index(drop=True)
        df_cut_valid = validate_plan(df_cut, material=material)

        # ④ 再展示【Machining-only Plan】
        display(Markdown("###  Machining-only Plan"))
        display_plan_table(df_cut_valid)

        # ⑤ 反思摘要基于完整流程（也可换成 df_cut_valid）
        reflect_summary(raw_json, df_full_valid)


In [10]:
# === 5. 按钮事件绑定 ===
generate_button.on_click(on_generate_clicked)

# === 6. 显示UI（必须放最后） ===
output_area = widgets.Output()
display(
    Markdown("## CNC Process Planner"),
    desc_input,
    material_selector,
    generate_button,
    output_area
)

## CNC Process Planner

Textarea(value='An aluminum gear with 20 teeth and a central bore.', description='Part:', layout=Layout(height…

Dropdown(description='Material:', layout=Layout(width='30%'), options=('aluminum', 'steel'), value='aluminum')

Button(button_style='success', description=' Generate CNC Plan', layout=Layout(margin='10px 0', width='30%'), …

Output()